In [ ]:
import pandas as pd
import re
import tldextract
# import tensorflow as tf
import numpy as np
import unicodedata
import matplotlib.pyplot as plt

nRowsRead = None  # specify 'None' if want to read whole file
postStrings = []
typeStrings = []


In [ ]:
# df.to_csv(path_or_buf="./csv_files/df_spojeni.csv")
# df = pd.read_csv(filepath_or_buffer="./csv_files/df_spojeni.csv")
# df=df.drop([3559])

df = pd.read_pickle("../neuralNet/pickle_files/df_odvojeni_word_vectors.pkl")
df = df.groupby(['type', 'avgWordLen'], sort=False,as_index=False).agg({'posts':' '.join,'upperCount':'first','stopWordCount':'first','urlCount':'first','wordCount':'first','avgWordLen':'first','vectors':lambda x:np.array(x)})


In [ ]:
df['type'].value_counts()

In [ ]:
df0 = df[df['type'] == 0].sample(450)
df1 = df[df['type'] == 1].sample(450)
df2 = df[df['type'] == 2].sample(450)
df3 = df[df['type'] == 3].sample(450)

dfr = pd.concat([df0,df1,df2,df3],)

In [ ]:
y = dfr['type']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(dtype=np.uint8)
text_counts = cv.fit_transform(dfr['posts'])

In [ ]:
dfr_bow = pd.DataFrame(text_counts.toarray(),columns=cv.get_feature_names_out())

In [ ]:
dfr_bow.head(2)

In [ ]:
print("Num. of words: " + str(len(list(dfr_bow.columns))))

In [ ]:
dfr_features = dfr.drop(labels=['posts','type','vectors'],axis=1).reset_index(drop=True)  

## ML ALGORITHMS

In [ ]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
import seaborn as sns

In [ ]:
rfc = []
rfc.append(RandomForestClassifier(random_state=42,n_estimators=100,n_jobs=-1))
rfc.append(RandomForestClassifier(random_state=42,n_estimators=200,n_jobs=-1))
rfc.append(RandomForestClassifier(random_state=42,n_estimators=500,n_jobs=-1))
rfc.append(RandomForestClassifier(random_state=42,n_estimators=1000,n_jobs=-1))
rfc.append(RandomForestClassifier(random_state=42,n_estimators=2000,n_jobs=-1))
rfc.append(RandomForestClassifier(random_state=42,n_estimators=10000,n_jobs=-1))

In [131]:
%%time

X = dfr_features.join(dfr_bow)

def classify(X,y,classifier):
  scaler = MinMaxScaler(feature_range=(0,1))
  X=scaler.fit_transform(X)

  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size= 0.2,random_state=69,stratify=y)

  classifier.fit(X_train,y_train)
  y_pred = classifier.predict(X_test)
  ac = accuracy_score(y_test,y_pred)

  # print("accuracy: ", ac)
  print(classification_report(y_test, y_pred))
  print("##################################################")

for classifier in rfc:
  classify(X,y,classifier)

              precision    recall  f1-score   support

           0       0.60      0.53      0.56        90
           1       0.57      0.61      0.59        90
           2       0.53      0.53      0.53        90
           3       0.59      0.61      0.60        90

    accuracy                           0.57       360
   macro avg       0.57      0.57      0.57       360
weighted avg       0.57      0.57      0.57       360

##################################################
              precision    recall  f1-score   support

           0       0.65      0.54      0.59        90
           1       0.58      0.66      0.61        90
           2       0.61      0.56      0.58        90
           3       0.60      0.68      0.64        90

    accuracy                           0.61       360
   macro avg       0.61      0.61      0.61       360
weighted avg       0.61      0.61      0.61       360

##################################################
              precision    r

### average accuracy on 100 forests

In [132]:
# def classify(X,y):
#   scaler = MinMaxScaler(feature_range=(0,1))
#   X=scaler.fit_transform(X)

#   X_train, X_test, y_train, y_test = train_test_split(X,y,test_size= 0.2,random_state=42,stratify=y)

#   rfc.fit(X_train,y_train)
#   y_pred = rfc.predict(X_test)
#   ac = accuracy_score(y_test,y_pred)

#   return ac

# test = []

# for i in range(0,100):
#   df0 = df[df['type'] == 0].sample(450)
#   df1 = df[df['type'] == 1].sample(450)
#   df2 = df[df['type'] == 2].sample(450)
#   df3 = df[df['type'] == 3].sample(450)
#   dfr = pd.concat([df0, df1, df2, df3],)
#   y = dfr['type']
#   cv = CountVectorizer(dtype='b')
#   text_counts = cv.fit_transform(dfr['posts'])
#   dfr_bow = pd.DataFrame(text_counts.toarray(),columns=cv.get_feature_names_out())
#   dfr_features = dfr.drop(labels=['posts','type','vectors'],axis=1).reset_index(drop=True)  

#   test.append(classify(dfr_features.join(dfr_bow), y))

# print(sum(test)/len(test))

In [133]:
# import statistics
# print("max= "+str(max(test)))
# print("min= "+str(min(test)))